## Summary


---

## Imports

In [1]:
from pathlib import Path

import numpy as np
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import matplotlib.pyplot as plt
from scipy import stats

In [2]:
pd.set_option("max_columns", 1000)

## Parameters

In [3]:
NOTEBOOK_NAME = "01_prepare_combined_dataset"
NOTEBOOK_PATH = Path(NOTEBOOK_NAME).resolve()
NOTEBOOK_PATH.mkdir(exist_ok=True)
NOTEBOOK_NAME

'01_prepare_combined_dataset'

## Load data

### Screen

In [4]:
screen_df = pd.read_excel("../input/asthma-study-2019.xlsx", "Screen")
assert len(screen_df["id"].drop_duplicates() == len(screen_df))

screen_df.head(2)

,id,date,clinic,infconsent,18+,eos300,eos150,sputum,acq-screen,result
0,2311,2017-02-11,23,y,y,0.0,NaN,NaN,3.666667,n
1,2800,2017-02-11,28,y,y,1.0,NaN,NaN,3.000000,NaN


### Demographics

In [5]:
demographics_0_df = pd.read_excel("../input/asthma-study-2019.xlsx", "Demographics")
demographics_0_df["demographics_table"] = 0

demographics_0_df["race"] = np.nan
del demographics_0_df["ethnicity"]

demographics_0_df.head(2)

,id,age,race,gender,work,retire,disab,demographics_table
0,102,25,NaN,0,1,NaN,0,0
1,105,97,NaN,0,0,0.0,0,0


In [6]:
demographics_1_df = pd.read_excel("../input/asthma-study-2019.xlsx", "Demographics(1)")
demographics_1_df["demographics_table"] = 1

demographics_1_df["age"] = (
    demographics_1_df["assess"].dt.year - demographics_1_df["birthyear"]
)
demographics_1_df = demographics_1_df.drop(pd.Index(["birthyear", "assess"]), axis=1)

demographics_1_df.head(2)

,id,race,sex,work,retire,disab,demographics_table,age
0,1113,filipino,0,0,1.0,0,1,41
1,2902,chinese,0,0,1.0,0,1,53


In [7]:
demographics_df = pd.concat([demographics_0_df, demographics_1_df], sort=False)

demographics_df.head(2)

,id,age,race,gender,work,retire,disab,demographics_table,sex
0,102,25,NaN,0.0,1,NaN,0,0,NaN
1,105,97,NaN,0.0,0,0.0,0,0,NaN


In [8]:
demographics_df.describe()

,id,age,gender,work,retire,disab,demographics_table,sex
count,401.000000,401.000000,57.000000,401.000000,204.000000,401.000000,401.000000,344.000000
mean,1461.788030,54.431421,1.315789,0.491272,0.534314,0.516209,0.857855,0.491279
std,951.628408,21.425823,1.020467,0.500548,0.500048,0.500361,0.349635,0.500652
min,100.000000,16.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,601.000000,36.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
50%,1410.000000,54.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000
75%,2300.000000,74.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000
max,3107.000000,99.000000,3.000000,1.000000,1.000000,1.000000,1.000000,1.000000


### Medical history

In [9]:
medhistory_0_df = pd.read_excel("../input/asthma-study-2019.xlsx", "medhistory")
medhistory_0_df["medhistory_table"] = 0

medhistory_0_df.head(2)

,id,co-copd,co-heartfailure,co-diabetes,co-yellowfever,co-asthma,co-pulmonaryembolism,co-bronchitis,co-arthritis,co-pneumonia,medhistory_table
0,100,NaN,0.511612,0.797143,NaN,0.789300,NaN,NaN,NaN,0.860042,0
1,101,NaN,NaN,0.730094,NaN,0.891995,NaN,NaN,NaN,0.442976,0


In [10]:
medhistory_1_df = pd.read_excel("../input/asthma-study-2019.xlsx", "medhistory(1)")
medhistory_1_df["medhistory_table"] = 1

medhistory_1_df["bmi"] = medhistory_1_df["weight"] / (medhistory_1_df["height"] ** 2)
medhistory_1_df.head(2)

,id,assess,smoking,polyps,vaccine,arthritis,CVD,Prev-biologic,allergy,weight,height,medhistory_table,bmi
0,134,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN
1,135,43689.0,0.0,0.4,0.08,0.31,0.07,0.8,0.2,112.0,55.0,1,0.037025


In [11]:
medhistory_df = pd.concat([medhistory_0_df, medhistory_1_df], sort=False)
medhistory_df.columns = [c.strip() for c in medhistory_df.columns]

medhistory_df.head(2)

,id,co-copd,co-heartfailure,co-diabetes,co-yellowfever,co-asthma,co-pulmonaryembolism,co-bronchitis,co-arthritis,co-pneumonia,medhistory_table,assess,smoking,polyps,vaccine,arthritis,CVD,Prev-biologic,allergy,weight,height,bmi
0,100,NaN,0.511612,0.797143,NaN,0.789300,NaN,NaN,NaN,0.860042,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,101,NaN,NaN,0.730094,NaN,0.891995,NaN,NaN,NaN,0.442976,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
medhistory_df.describe()

,id,co-copd,co-heartfailure,co-diabetes,co-yellowfever,co-asthma,co-pulmonaryembolism,co-bronchitis,co-arthritis,co-pneumonia,medhistory_table,assess,smoking,polyps,vaccine,arthritis,CVD,Prev-biologic,allergy,weight,height,bmi
count,401.000000,34.000000,45.000000,44.000000,1.0,42.000000,0.0,2.0,21.000000,50.000000,401.000000,286.000000,286.000000,286.000000,286.000000,286.000000,286.000000,286.000000,286.000000,286.000000,286.000000,286.000000
mean,1461.788030,0.352941,0.526819,0.534584,1.0,0.703904,NaN,1.0,0.666667,0.516479,0.857855,43247.195804,0.238901,0.287762,0.257203,0.242902,0.372867,0.617832,0.418881,208.863636,64.090909,0.053890
std,951.628408,0.485071,0.286956,0.279710,NaN,0.301220,NaN,0.0,0.483046,0.309451,0.349635,282.103534,0.235883,0.201983,0.163741,0.143212,0.222661,0.303425,0.286161,74.412216,9.733941,0.023620
min,100.000000,0.000000,0.003784,0.013703,1.0,0.079592,NaN,1.0,0.000000,0.018830,0.000000,42777.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,10.000000,48.000000,0.002296
25%,601.000000,0.000000,0.287912,0.357410,1.0,0.585406,NaN,1.0,0.000000,0.243446,1.000000,43000.250000,0.000000,0.100000,0.110000,0.130000,0.180000,0.400000,0.200000,153.250000,56.000000,0.036587
50%,1410.000000,0.000000,0.511612,0.581149,1.0,0.806314,NaN,1.0,1.000000,0.499649,1.000000,43242.000000,0.188894,0.300000,0.250000,0.235000,0.390000,0.600000,0.400000,208.000000,64.000000,0.049035
75%,2300.000000,1.000000,0.751549,0.758209,1.0,0.929266,NaN,1.0,1.000000,0.844424,1.000000,43472.750000,0.432714,0.500000,0.390000,0.357500,0.570000,0.950000,0.700000,259.000000,73.000000,0.067101
max,3107.000000,1.000000,0.984069,0.971709,1.0,1.000000,NaN,1.0,1.000000,0.996887,1.000000,43753.000000,0.890896,0.600000,0.550000,0.510000,0.740000,1.000000,0.900000,866.000000,80.000000,0.149931


### ACQ

In [13]:
acq_df = pd.read_excel("../input/asthma-study-2019.xlsx", "ACQ")

acq_df.head(2)

,id,date,acq,date2,acq2,date3,acq3,date4,acq4,date5,acq5
0,122,1900-01-01,3.666667,1900-01-02,2.200000,1900-01-03,2.420000,1900-01-04,2.420,1900-01-05,2.1780
1,129,1900-01-01,1.833333,1900-01-02,0.366667,1900-01-03,0.403333,2018-11-28,0.484,2019-02-07,0.4356


## Combine

In [14]:
combined_df = (
    acq_df
    .merge(medhistory_df, on=["id"], how="outer", validate="1:1")
    .merge(demographics_df, on=["id"], how="outer", validate="1:1")
    .merge(screen_df, on=["id"], how="outer", validate="1:1")
)

assert len(combined_df) == len(screen_df)

In [15]:
combined_df.describe()

,id,acq,acq2,acq3,acq4,acq5,co-copd,co-heartfailure,co-diabetes,co-yellowfever,co-asthma,co-pulmonaryembolism,co-bronchitis,co-arthritis,co-pneumonia,medhistory_table,assess,smoking,polyps,vaccine,arthritis,CVD,Prev-biologic,allergy,weight,height,bmi,age,gender,work,retire,disab,demographics_table,sex,clinic,eos300,eos150,sputum,acq-screen
count,401.000000,197.000000,192.000000,194.000000,193.000000,177.000000,34.000000,45.000000,44.000000,1.0,42.000000,0.0,2.0,21.000000,50.000000,401.000000,286.000000,286.000000,286.000000,286.000000,286.000000,286.000000,286.000000,286.000000,286.000000,286.000000,286.000000,401.000000,57.000000,401.000000,204.000000,401.000000,401.000000,344.000000,401.000000,379.000000,0.0,0.0,401.000000
mean,1461.788030,3.962775,3.075521,2.928076,2.931402,2.291444,0.352941,0.526819,0.534584,1.0,0.703904,NaN,1.0,0.666667,0.516479,0.857855,43247.195804,0.238901,0.287762,0.257203,0.242902,0.372867,0.617832,0.418881,208.863636,64.090909,0.053890,54.431421,1.315789,0.491272,0.534314,0.516209,0.857855,0.491279,14.543641,0.535620,NaN,NaN,3.596010
std,951.628408,2.297162,2.943295,2.860069,2.985490,1.837040,0.485071,0.286956,0.279710,NaN,0.301220,NaN,0.0,0.483046,0.309451,0.349635,282.103534,0.235883,0.201983,0.163741,0.143212,0.222661,0.303425,0.286161,74.412216,9.733941,0.023620,21.425823,1.020467,0.500548,0.500048,0.500361,0.349635,0.500652,9.540111,0.499389,NaN,NaN,1.978529
min,100.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.003784,0.013703,1.0,0.079592,NaN,1.0,0.000000,0.018830,0.000000,42777.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,10.000000,48.000000,0.002296,16.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,NaN,NaN,0.000000
25%,601.000000,2.500000,0.962500,1.000000,0.968000,0.950400,0.000000,0.287912,0.357410,1.0,0.585406,NaN,1.0,0.000000,0.243446,1.000000,43000.250000,0.000000,0.100000,0.110000,0.130000,0.180000,0.400000,0.200000,153.250000,56.000000,0.036587,36.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,6.000000,0.000000,NaN,NaN,2.166667
50%,1410.000000,3.500000,2.100000,2.122500,2.024000,1.887600,0.000000,0.511612,0.581149,1.0,0.806314,NaN,1.0,1.000000,0.499649,1.000000,43242.000000,0.188894,0.300000,0.250000,0.235000,0.390000,0.600000,0.400000,208.000000,64.000000,0.049035,54.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,14.000000,1.000000,NaN,NaN,3.500000
75%,2300.000000,5.000000,4.229167,4.075000,3.813333,3.373333,1.000000,0.751549,0.758209,1.0,0.929266,NaN,1.0,1.000000,0.844424,1.000000,43472.750000,0.432714,0.500000,0.390000,0.357500,0.570000,0.950000,0.700000,259.000000,73.000000,0.067101,74.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,23.000000,1.000000,NaN,NaN,4.666667
max,3107.000000,15.000000,11.000000,15.000000,16.000000,9.856000,1.000000,0.984069,0.971709,1.0,1.000000,NaN,1.0,1.000000,0.996887,1.000000,43753.000000,0.890896,0.600000,0.550000,0.510000,0.740000,1.000000,0.900000,866.000000,80.000000,0.149931,99.000000,3.000000,1.000000,1.000000,1.000000,1.000000,1.000000,31.000000,1.000000,NaN,NaN,15.000000


## Write output

In [16]:
table = pa.Table.from_pandas(combined_df, preserve_index=False)
pq.write_table(table, NOTEBOOK_PATH.joinpath("combined.parquet"))

In [ ]:
 datetime.date()

In [36]:
from datetime import datetime

(acq_df[["date", "date2", "date3", "date4", "date5"]] > datetime.now()).any(axis=1).sum()

18

In [18]:
(acq_df[["acq", "acq2", "acq3", "acq4", "acq5"]] > 6).any(axis=1).sum()

29

In [37]:
set(demographics_0_df["id"]) & set(demographics_1_df["id"])

set()

In [40]:
set(medhistory_0_df["id "]) & set(medhistory_1_df["id "])

set()

In [17]:
combined_df.corr()["acq"]

id                      0.018162
acq                     1.000000
acq2                    0.549578
acq3                    0.741947
acq4                    0.759623
acq5                    0.228255
co-copd                 0.140659
co-heartfailure        -0.172237
co-diabetes            -0.148079
co-yellowfever               NaN
co-asthma               0.148670
co-pulmonaryembolism         NaN
co-bronchitis                NaN
co-arthritis           -0.082296
co-pneumonia            0.320265
medhistory_table        0.072209
assess                  0.092634
smoking                 0.084104
polyps                  0.089507
vaccine                 0.109803
arthritis               0.049442
CVD                     0.058923
Prev-biologic           0.042100
allergy                 0.103675
weight                  0.023647
height                  0.017043
bmi                     0.010844
age                     0.028466
gender                  0.100988
work                   -0.116199
retire    